# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 128          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)
print(f"vocab_size is {vocab_size}")

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.84s)
creating index...


  0%|          | 388/414113 [00:00<01:46, 3877.60it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:25<00:00, 4856.29it/s]


vocab_size is 8855


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:01<00:00, 57031578.81it/s]


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [2]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

torch.Size([128, 256])
torch.Size([128, 12])
Epoch [1/3], Step [1/3236], Loss: 9.1040, Perplexity: 8991.0407torch.Size([128, 256])
torch.Size([128, 10])
Epoch [1/3], Step [2/3236], Loss: 8.0689, Perplexity: 3193.4685torch.Size([128, 256])
torch.Size([128, 16])
Epoch [1/3], Step [3/3236], Loss: 7.7664, Perplexity: 2359.9812torch.Size([128, 256])
torch.Size([128, 12])
Epoch [1/3], Step [4/3236], Loss: 5.9648, Perplexity: 389.4862torch.Size([128, 256])
torch.Size([128, 12])
Epoch [1/3], Step [5/3236], Loss: 5.5645, Perplexity: 261.0007torch.Size([128, 256])
torch.Size([128, 12])
Epoch [1/3], Step [6/3236], Loss: 5.3168, Perplexity: 203.7288torch.Size([128, 256])
torch.Size([128, 14])
Epoch [1/3], Step [7/3236], Loss: 5.3133, Perplexity: 203.0252torch.Size([128, 256])
torch.Size([128, 13])
Epoch [1/3], Step [8/3236], Loss: 5.1667, Perplexity: 175.3424torch.Size([128, 256])
torch.Size([128, 12])
Epoch [1/3], Step [9/3236], Loss: 5.1430, Perplexity: 171.2370torch.Size([128, 256])
torch.Size(

Epoch [1/3], Step [153/3236], Loss: 3.1847, Perplexity: 24.1599torch.Size([128, 256])
torch.Size([128, 11])
Epoch [1/3], Step [154/3236], Loss: 3.3391, Perplexity: 28.1946torch.Size([128, 256])
torch.Size([128, 11])
Epoch [1/3], Step [155/3236], Loss: 3.3971, Perplexity: 29.8765torch.Size([128, 256])
torch.Size([128, 18])
Epoch [1/3], Step [156/3236], Loss: 3.4775, Perplexity: 32.3787torch.Size([128, 256])
torch.Size([128, 12])
Epoch [1/3], Step [157/3236], Loss: 3.1220, Perplexity: 22.6910torch.Size([128, 256])
torch.Size([128, 10])
Epoch [1/3], Step [158/3236], Loss: 3.6677, Perplexity: 39.1622torch.Size([128, 256])
torch.Size([128, 20])
Epoch [1/3], Step [159/3236], Loss: 3.6544, Perplexity: 38.6444torch.Size([128, 256])
torch.Size([128, 11])
Epoch [1/3], Step [160/3236], Loss: 3.2821, Perplexity: 26.6319torch.Size([128, 256])
torch.Size([128, 19])
Epoch [1/3], Step [161/3236], Loss: 3.6688, Perplexity: 39.2036torch.Size([128, 256])
torch.Size([128, 20])
Epoch [1/3], Step [162/3236]

Epoch [1/3], Step [229/3236], Loss: 3.0973, Perplexity: 22.1381torch.Size([128, 256])
torch.Size([128, 9])
Epoch [1/3], Step [230/3236], Loss: 3.7926, Perplexity: 44.3737torch.Size([128, 256])
torch.Size([128, 12])
Epoch [1/3], Step [231/3236], Loss: 3.2748, Perplexity: 26.4381torch.Size([128, 256])
torch.Size([128, 12])
Epoch [1/3], Step [232/3236], Loss: 3.1340, Perplexity: 22.9660torch.Size([128, 256])
torch.Size([128, 12])
Epoch [1/3], Step [233/3236], Loss: 3.0797, Perplexity: 21.7510torch.Size([128, 256])
torch.Size([128, 12])
Epoch [1/3], Step [234/3236], Loss: 3.0644, Perplexity: 21.4214torch.Size([128, 256])
torch.Size([128, 15])
Epoch [1/3], Step [235/3236], Loss: 3.2038, Perplexity: 24.6266torch.Size([128, 256])
torch.Size([128, 13])
Epoch [1/3], Step [236/3236], Loss: 3.1820, Perplexity: 24.0946torch.Size([128, 256])
torch.Size([128, 11])
Epoch [1/3], Step [237/3236], Loss: 3.3261, Perplexity: 27.8307torch.Size([128, 256])
torch.Size([128, 15])
Epoch [1/3], Step [238/3236],

Epoch [1/3], Step [305/3236], Loss: 3.6388, Perplexity: 38.0449torch.Size([128, 256])
torch.Size([128, 16])
Epoch [1/3], Step [306/3236], Loss: 3.2246, Perplexity: 25.1424torch.Size([128, 256])
torch.Size([128, 11])
Epoch [1/3], Step [307/3236], Loss: 3.1969, Perplexity: 24.4578torch.Size([128, 256])
torch.Size([128, 14])
Epoch [1/3], Step [308/3236], Loss: 3.1779, Perplexity: 23.9971torch.Size([128, 256])
torch.Size([128, 16])
Epoch [1/3], Step [309/3236], Loss: 3.1974, Perplexity: 24.4679torch.Size([128, 256])
torch.Size([128, 12])
Epoch [1/3], Step [310/3236], Loss: 3.1455, Perplexity: 23.2304torch.Size([128, 256])
torch.Size([128, 11])
Epoch [1/3], Step [311/3236], Loss: 3.1115, Perplexity: 22.4553torch.Size([128, 256])
torch.Size([128, 13])
Epoch [1/3], Step [312/3236], Loss: 2.9931, Perplexity: 19.9472torch.Size([128, 256])
torch.Size([128, 11])
Epoch [1/3], Step [313/3236], Loss: 3.1888, Perplexity: 24.2600torch.Size([128, 256])
torch.Size([128, 15])
Epoch [1/3], Step [314/3236]

Epoch [1/3], Step [381/3236], Loss: 3.0840, Perplexity: 21.8455torch.Size([128, 256])
torch.Size([128, 11])
Epoch [1/3], Step [382/3236], Loss: 3.1484, Perplexity: 23.2977torch.Size([128, 256])
torch.Size([128, 11])
Epoch [1/3], Step [383/3236], Loss: 3.1327, Perplexity: 22.9360torch.Size([128, 256])
torch.Size([128, 13])
Epoch [1/3], Step [384/3236], Loss: 3.0151, Perplexity: 20.3908torch.Size([128, 256])
torch.Size([128, 12])
Epoch [1/3], Step [385/3236], Loss: 3.1522, Perplexity: 23.3866torch.Size([128, 256])
torch.Size([128, 13])
Epoch [1/3], Step [386/3236], Loss: 3.0991, Perplexity: 22.1772torch.Size([128, 256])
torch.Size([128, 13])
Epoch [1/3], Step [387/3236], Loss: 3.0531, Perplexity: 21.1808torch.Size([128, 256])
torch.Size([128, 12])
Epoch [1/3], Step [388/3236], Loss: 3.0805, Perplexity: 21.7682torch.Size([128, 256])
torch.Size([128, 14])
Epoch [1/3], Step [389/3236], Loss: 3.0665, Perplexity: 21.4660torch.Size([128, 256])
torch.Size([128, 12])
Epoch [1/3], Step [390/3236]

...

Epoch [3/3], Step [2495/3236], Loss: 2.7480, Perplexity: 15.6118torch.Size([128, 256])
torch.Size([128, 13])
Epoch [3/3], Step [2496/3236], Loss: 2.9136, Perplexity: 18.4221torch.Size([128, 256])
torch.Size([128, 13])
Epoch [3/3], Step [2497/3236], Loss: 2.8484, Perplexity: 17.2608torch.Size([128, 256])
torch.Size([128, 13])
Epoch [3/3], Step [2498/3236], Loss: 2.8581, Perplexity: 17.4292torch.Size([128, 256])
torch.Size([128, 12])
Epoch [3/3], Step [2499/3236], Loss: 2.7892, Perplexity: 16.2677torch.Size([128, 256])
torch.Size([128, 21])
Epoch [3/3], Step [2500/3236], Loss: 3.4051, Perplexity: 30.1185
torch.Size([128, 256])
torch.Size([128, 12])
Epoch [3/3], Step [2501/3236], Loss: 2.8523, Perplexity: 17.3276torch.Size([128, 256])
torch.Size([128, 14])
Epoch [3/3], Step [2502/3236], Loss: 2.9037, Perplexity: 18.2407torch.Size([128, 256])
torch.Size([128, 11])
Epoch [3/3], Step [2503/3236], Loss: 2.8571, Perplexity: 17.4112torch.Size([128, 256])
torch.Size([128, 11])
Epoch [3/3], Step 

Epoch [3/3], Step [2645/3236], Loss: 2.8757, Perplexity: 17.7374torch.Size([128, 256])
torch.Size([128, 11])
Epoch [3/3], Step [2646/3236], Loss: 2.7355, Perplexity: 15.4167torch.Size([128, 256])
torch.Size([128, 18])
Epoch [3/3], Step [2647/3236], Loss: 3.1435, Perplexity: 23.1855torch.Size([128, 256])
torch.Size([128, 15])
Epoch [3/3], Step [2648/3236], Loss: 3.0096, Perplexity: 20.2803torch.Size([128, 256])
torch.Size([128, 13])
Epoch [3/3], Step [2649/3236], Loss: 3.0494, Perplexity: 21.1023torch.Size([128, 256])
torch.Size([128, 13])
Epoch [3/3], Step [2650/3236], Loss: 2.8344, Perplexity: 17.0198torch.Size([128, 256])
torch.Size([128, 13])
Epoch [3/3], Step [2651/3236], Loss: 2.8744, Perplexity: 17.7140torch.Size([128, 256])
torch.Size([128, 11])
Epoch [3/3], Step [2652/3236], Loss: 2.7384, Perplexity: 15.4625torch.Size([128, 256])
torch.Size([128, 11])
Epoch [3/3], Step [2653/3236], Loss: 2.9996, Perplexity: 20.0778torch.Size([128, 256])
torch.Size([128, 13])
Epoch [3/3], Step [

Epoch [3/3], Step [2795/3236], Loss: 3.0369, Perplexity: 20.8409torch.Size([128, 256])
torch.Size([128, 12])
Epoch [3/3], Step [2796/3236], Loss: 2.8180, Perplexity: 16.7432torch.Size([128, 256])
torch.Size([128, 16])
Epoch [3/3], Step [2797/3236], Loss: 3.0654, Perplexity: 21.4439torch.Size([128, 256])
torch.Size([128, 14])
Epoch [3/3], Step [2798/3236], Loss: 2.8885, Perplexity: 17.9661torch.Size([128, 256])
torch.Size([128, 14])
Epoch [3/3], Step [2799/3236], Loss: 2.9488, Perplexity: 19.0829torch.Size([128, 256])
torch.Size([128, 12])
Epoch [3/3], Step [2800/3236], Loss: 2.9515, Perplexity: 19.1343
torch.Size([128, 256])
torch.Size([128, 15])
Epoch [3/3], Step [2801/3236], Loss: 2.9186, Perplexity: 18.5146torch.Size([128, 256])
torch.Size([128, 11])
Epoch [3/3], Step [2802/3236], Loss: 2.8890, Perplexity: 17.9757torch.Size([128, 256])
torch.Size([128, 12])
Epoch [3/3], Step [2803/3236], Loss: 2.7622, Perplexity: 15.8340torch.Size([128, 256])
torch.Size([128, 13])
Epoch [3/3], Step 

Epoch [3/3], Step [2945/3236], Loss: 2.8805, Perplexity: 17.8230torch.Size([128, 256])
torch.Size([128, 10])
Epoch [3/3], Step [2946/3236], Loss: 2.9895, Perplexity: 19.8761torch.Size([128, 256])
torch.Size([128, 13])
Epoch [3/3], Step [2947/3236], Loss: 2.8466, Perplexity: 17.2283torch.Size([128, 256])
torch.Size([128, 14])
Epoch [3/3], Step [2948/3236], Loss: 2.7871, Perplexity: 16.2337torch.Size([128, 256])
torch.Size([128, 13])
Epoch [3/3], Step [2949/3236], Loss: 2.9016, Perplexity: 18.2031torch.Size([128, 256])
torch.Size([128, 11])
Epoch [3/3], Step [2950/3236], Loss: 2.9384, Perplexity: 18.8848torch.Size([128, 256])
torch.Size([128, 13])
Epoch [3/3], Step [2951/3236], Loss: 2.9026, Perplexity: 18.2221torch.Size([128, 256])
torch.Size([128, 15])
Epoch [3/3], Step [2952/3236], Loss: 3.0684, Perplexity: 21.5072torch.Size([128, 256])
torch.Size([128, 11])
Epoch [3/3], Step [2953/3236], Loss: 2.8820, Perplexity: 17.8505torch.Size([128, 256])
torch.Size([128, 15])
Epoch [3/3], Step [

Epoch [3/3], Step [3095/3236], Loss: 2.8595, Perplexity: 17.4524torch.Size([128, 256])
torch.Size([128, 11])
Epoch [3/3], Step [3096/3236], Loss: 2.9082, Perplexity: 18.3239torch.Size([128, 256])
torch.Size([128, 10])
Epoch [3/3], Step [3097/3236], Loss: 2.9959, Perplexity: 20.0031torch.Size([128, 256])
torch.Size([128, 11])
Epoch [3/3], Step [3098/3236], Loss: 2.9179, Perplexity: 18.5023torch.Size([128, 256])
torch.Size([128, 16])
Epoch [3/3], Step [3099/3236], Loss: 3.1228, Perplexity: 22.7089torch.Size([128, 256])
torch.Size([128, 11])
Epoch [3/3], Step [3100/3236], Loss: 2.9328, Perplexity: 18.7800
torch.Size([128, 256])
torch.Size([128, 13])
Epoch [3/3], Step [3101/3236], Loss: 2.8119, Perplexity: 16.6419torch.Size([128, 256])
torch.Size([128, 11])
Epoch [3/3], Step [3102/3236], Loss: 2.8000, Perplexity: 16.4442torch.Size([128, 256])
torch.Size([128, 11])
Epoch [3/3], Step [3103/3236], Loss: 2.8009, Perplexity: 16.4592torch.Size([128, 256])
torch.Size([128, 11])
Epoch [3/3], Step 

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [3]:
# (Optional) TODO: Validate your model.